In [1]:
import random,math,copy,timeit,operator
from sklearn import utils,preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from copy import deepcopy as dc
from sys import exit
from sklearn import svm
from sklearn import model_selection as ms

In [2]:
class ants(object):
    def __init__(self,gens=None,mse=None):
        self.gens=gens
        self.mse=mse

In [3]:
def random_search(n,dim):
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens



In [4]:
def k_search(n,dim,flist,pheromone):
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
        for j,x in enumerate(gen):
            gen[j]=flist[j]*gen[j]
    return gens


In [5]:
def evaluate(train_d,train_l,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in train_d])
        kf = ms.StratifiedKFold(n_splits=4)
        s = 0
        for tr_ix,te_ix in kf.split(al_data, train_l):
            s+= svm.LinearSVC().fit(al_data[tr_ix],train_l[tr_ix]).score(al_data[te_ix],train_l[te_ix])#.predict(al_test_data)
        s/=4
        return s

In [6]:
def evaluate(train_d,train_l,gen):
        mask = np.array(gen) > 0
        # if no features selected, return worst/zero score to avoid classifier errors
        if np.sum(mask) == 0:
            return 0.0
        al_data = np.array([al[mask] for al in train_d])
        kf = ms.StratifiedKFold(n_splits=4)
        s = 0
        for tr_ix,te_ix in kf.split(al_data, train_l):
            try:
                s += svm.LinearSVC(max_iter=5000).fit(al_data[tr_ix],train_l[tr_ix]).score(al_data[te_ix],train_l[te_ix])
            except Exception:
                # if classifier fails for any reason, treat as zero performance
                return 0.0
        s /= 4
        return s

'\ndef evaluate(train_d,train_l,gen):\n        mask = np.array(gen) > 0\n        # if no features selected, return worst/zero score to avoid classifier errors\n        if np.sum(mask) == 0:\n            return 0.0\n        al_data = np.array([al[mask] for al in train_d])\n        kf = ms.StratifiedKFold(n_splits=4)\n        s = 0\n        for tr_ix,te_ix in kf.split(al_data, train_l):\n            try:\n                s += svm.LinearSVC(max_iter=5000).fit(al_data[tr_ix],train_l[tr_ix]).score(al_data[te_ix],train_l[te_ix])\n            except Exception:\n                # if classifier fails for any reason, treat as zero performance\n                return 0.0\n        s /= 4\n        return s\n'

In [7]:
def BACO(train_d,train_l,n=30,max_iter=25,T0=0.1,k=10,m=25,p=3,rho=0.25):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            max_iter: Number of max iteration, default=300
            }
    output:{
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    dim=len(train_d[0])
    global_best=float("-inf")
    pheromone=list([T0]*dim)
    pher_diff=list([0]*dim)
    global_position=tuple([0]*dim)
    antslist=[]
    gens_dict = {}
    for i in range(n):
        gens=random_search(m,dim)
        mse=0
        for gen in gens:
            if tuple(gen) in gens_dict:
                score = gens_dict[tuple(gen)]
            else:
                score=evaluate(train_d,train_l,gen)
                gens_dict[tuple(gen)]=score
            global_best=score
            global_position=dc(gen)
            mse+=score
        mse=mse/m
        antslist.append(ants(gens,mse))
    antslist.sort(key=operator.attrgetter('mse'), reverse=True)
    klist=list(antslist[i] for i in range(k))
    mselist=list(antslist[i].mse for i in range(k))
    for j in range(len(klist)):
        diff = (max(mselist) - mselist[j]) / (max(mselist) - min(mselist) + 1e-9)
        flist=list([0]*dim)
        for gen in klist[j].gens:
            for a,x in enumerate(gen):
                if x==1:
                    pher_diff[a]=diff
                    flist[a]=1
                else:
                    pher_diff[a]=0
    
    for i in range(len(pheromone)):
        pheromone[i]=pheromone[i]*rho + pher_diff[i]
        
        
    for it in range(max_iter):
        m=m-p
        for i in range(n):
            gens=k_search(m,dim,flist,pheromone)
            mse=0
            for gen in gens:
                if tuple(gen) in gens_dict:
                    score = gens_dict[tuple(gen)]
                else:
                    score=evaluate(train_d,train_l,gen)
                    gens_dict[tuple(gen)]=score
                global_best=score
                global_position=dc(gen)
                mse+=score
            mse=mse/m
            antslist.append(ants(gens,mse))
        antslist.sort(key=operator.attrgetter('mse'), reverse=True)
        klist=list(antslist[i] for i in range(k))
        mselist=list(antslist[i].mse for i in range(k))
        for j in range(len(klist)):
            diff = (max(mselist) - mselist[j]) / (max(mselist) - min(mselist) + 1e-9)
            flist=list([0]*dim)
            for gen in klist[j].gens:
                for i,x in enumerate(gen):
                    if x==1:
                        pher_diff[i]=diff
                        flist[i]=1
                    else:
                        pher_diff[i]=0
    
        for i in range(len(pheromone)):
            pheromone[i]=pheromone[i]*rho + pher_diff[i]

        
    return global_position,global_position.count(1)


In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
import numpy as np

def test_score(gen, tr_x, tr_y, te_x, te_y):
    mask = np.array(gen) == 1
    al_data = np.array(tr_x[:, mask])
    al_test_data = np.array(te_x[:, mask])

    accuracies, precisions, recalls, f1s = [], [], [], []
    for _ in range(5):
        model = SVC(kernel='rbf', C=1.0, gamma='scale') 
        model.fit(al_data, tr_y)
        preds = model.predict(al_test_data)

        accuracies.append(accuracy_score(te_y, preds))
        precisions.append(precision_score(te_y, preds, zero_division=0))
        recalls.append(recall_score(te_y, preds, zero_division=0))
        f1s.append(f1_score(te_y, preds, zero_division=0))

    return (
        np.mean(accuracies),
        np.mean(precisions),
        np.mean(recalls),
        np.mean(f1s)
    )

In [9]:
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

with open("../../data/wdbc.data.csv") as f:
    lines = f.read().splitlines()

# id (columna 0), diagnosis (columna 1), resto son features numéricas
x = np.array([[float(d) for d in line.split(',')[2:]] for line in lines])  # desde la 3ra columna
y = np.array([line.split(',')[1] for line in lines])  # diagnosis ('M' o 'B')

lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)

train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)


print("Shape X:", x.shape)

Shape X: (569, 30)


In [ ]:
# ...existing code...
k = [1 for r in range(len(x[0]))]
print(test_score(k, train_d, train_l, test_d, test_l))

fattr = 0
# per-metric accumulators
ftest_acc = 0.0
ftest_prec = 0.0
ftest_rec = 0.0
ftest_f1 = 0.0

flist = [0 for _ in range(len(x[0]))]
final_list = [0 for _ in range(len(x[0]))]

start = timeit.default_timer()

for i in range(20):
    g, l = BACO(train_d, train_l, n=30, max_iter=25, T0=0.1, k=10, m=25, p=3, rho=0.25)
    
    # Asegurar que 'l' sea numérico
    if isinstance(l, list):
        l = len(l)
    elif not isinstance(l, (int, float)):
        l = float(l)

    fattr += l

    # unpack metrics
    acc, prec, rec, f1 = test_score(g, train_d, train_l, test_d, test_l)
    ftest_acc += acc
    ftest_prec += prec
    ftest_rec += rec
    ftest_f1 += f1
    
    for j in range(len(flist)):
        if g[j] == 1:
            flist[j] += 1

    # print per-iteration metrics (Acc and F1 shown)
    print(f"{i + 1:2d}  {''.join(map(str, g))}  {l:.4f}  Acc:{acc:.4f}  F1:{f1:.4f}")

# Promedios
fattr /= 20
ftest_acc /= 20
ftest_prec /= 20
ftest_rec /= 20
ftest_f1 /= 20

end = timeit.default_timer()
time = end - start

# Asegurar que fattr esté dentro de rango y sea entero
final_count = int(round(fattr))
final_count = max(0, min(final_count, len(flist)))

final = np.argsort(flist)[::-1][:final_count]

for idx in final:
    final_list[idx] = 1

print(f"Final:  {''.join(map(str, final_list))}   {fattr:.4f}   Acc:{ftest_acc:.6f}  F1:{ftest_f1:.6f}    {time:.4f}")
# ...existing code...

(np.float64(0.944055944055944), np.float64(0.9811320754716981), np.float64(0.8813559322033898), np.float64(0.9285714285714286))
 1  111111111111111111111111111111  30.0000  Acc:0.9441  F1:0.9286
 2  100100010000001000000000011000  6.0000  Acc:0.8951  F1:0.8598
 3  100111001111101011111011010111  21.0000  Acc:0.9441  F1:0.9286
 4  111111101011111111111111111111  28.0000  Acc:0.9441  F1:0.9286
 5  000000000001011000010001010110  8.0000  Acc:0.9441  F1:0.9298
 6  111111100100110011110110100110  19.0000  Acc:0.9091  F1:0.8807
 7  001000111110011100010010101000  13.0000  Acc:0.9580  F1:0.9464
 8  100011110111101111011111011111  23.0000  Acc:0.9441  F1:0.9298
 9  011001111111111110010011100110  20.0000  Acc:0.9441  F1:0.9298
10  111111110111111011111111111111  28.0000  Acc:0.9441  F1:0.9286
11  100001111101110011100101101100  17.0000  Acc:0.9441  F1:0.9298
